# Types

> Type definitions and protocols for the BYOK system

In [ ]:
#| default_exp core.types

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Dict, Any, Protocol, runtime_checkable
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from enum import Enum

## Storage Types

In [ ]:
#| export
class StorageBackend(Enum):
    """Available storage backends for API keys"""
    SESSION = "session"  # Session-only storage
    DATABASE = "database"  # Database-backed storage
    HYBRID = "hybrid"  # Both session and database

In [ ]:
#| export
@dataclass
class APIKey:
    """Represents an encrypted API key with metadata"""
    provider: str  # e.g., 'openai', 'anthropic', 'google'
    encrypted_value: bytes  # Encrypted key value
    created_at: datetime = field(default_factory=datetime.now)
    expires_at: Optional[datetime] = None
    user_id: Optional[str] = None  # For database storage
    
    @property
    def is_expired(
        self
    ) -> bool:  # True if key has expired, False otherwise
        """Check if the key has expired"""
        if self.expires_at is None:
            return False
        return datetime.now() > self.expires_at
    
    def to_dict(
        self
    ) -> Dict[str, Any]:  # Dictionary representation for serialization
        """Convert to dictionary for storage"""
        return {
            'provider': self.provider,
            'encrypted_value': self.encrypted_value.hex(),
            'created_at': self.created_at.isoformat(),
            'expires_at': self.expires_at.isoformat() if self.expires_at else None,
            'user_id': self.user_id
        }
    
    @classmethod
    def from_dict(
        cls,  # The APIKey class
        data: Dict[str, Any]  # Dictionary containing serialized key data
    ) -> 'APIKey':  # Reconstructed APIKey instance
        """Create from dictionary"""
        return cls(
            provider=data['provider'],
            encrypted_value=bytes.fromhex(data['encrypted_value']),
            created_at=datetime.fromisoformat(data['created_at']),
            expires_at=datetime.fromisoformat(data['expires_at']) if data.get('expires_at') else None,
            user_id=data.get('user_id')
        )

## Storage Protocol

In [ ]:
#| export
@runtime_checkable
class KeyStorage(Protocol):
    """Protocol for key storage implementations"""
    
    def store(
        self,
        request: Any,  # FastHTML/Starlette request object
        key: APIKey  # API key object to store
    ) -> None:
        """Store an API key"""
        ...
    
    def retrieve(
        self,
        request: Any,  # FastHTML/Starlette request object
        provider: str,  # Provider name to retrieve key for
        user_id: Optional[str] = None  # Optional user ID for database lookup
    ) -> Optional[APIKey]:  # API key object if found, None otherwise
        """Retrieve an API key for a provider"""
        ...
    
    def delete(
        self,
        request: Any,  # FastHTML/Starlette request object
        provider: str,  # Provider name to delete key for
        user_id: Optional[str] = None  # Optional user ID for database deletion
    ) -> None:
        """Delete an API key"""
        ...
    
    def list_providers(
        self,
        request: Any,  # FastHTML/Starlette request object
        user_id: Optional[str] = None  # Optional user ID for database lookup
    ) -> list[str]:  # List of provider names with stored keys
        """List all stored providers"""
        ...
    
    def clear_all(
        self,
        request: Any,  # FastHTML/Starlette request object
        user_id: Optional[str] = None  # Optional user ID for database clearing
    ) -> None:
        """Clear all stored keys"""
        ...

## Configuration

In [ ]:
#| export
@dataclass
class BYOKConfig:
    """Configuration for the BYOK system"""
    storage_backend: StorageBackend = StorageBackend.SESSION
    encryption_key: Optional[bytes] = None  # If None, will be generated
    default_ttl: Optional[timedelta] = timedelta(hours=24)  # Default key expiration
    session_key_prefix: str = "byok_"  # Prefix for session storage keys
    db_table_name: str = "user_api_keys"  # Database table name
    auto_cleanup: bool = True  # Auto-cleanup expired keys
    require_https: bool = True  # Warn if not using HTTPS in production

## Database Schema

In [ ]:
#| export
class UserAPIKey:
    """Database schema for persistent API key storage (for use with fastsql)"""
    id: int
    user_id: str
    provider: str
    encrypted_value: str  # Hex-encoded encrypted value
    created_at: str  # ISO format datetime
    expires_at: str = ""  # ISO format datetime (empty string if no expiration)

## Exceptions

In [ ]:
#| export
class BYOKException(Exception):
    """Base exception for BYOK errors"""
    pass

class EncryptionError(BYOKException):
    """Error during encryption/decryption"""
    pass

class StorageError(BYOKException):
    """Error during storage operations"""
    pass

class KeyNotFoundError(BYOKException):
    """Requested key not found"""
    pass

class SecurityWarning(BYOKException):
    """Security-related warning"""
    pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()